# <hr style="clear: both" />

# Running Spark in YARN-client mode

This notebook demonstrates how to set up a SparkContext that uses SURFsara's Hadoop cluster: [YARN resourcemanager](http://head05.hathi.surfsara.nl:8088/cluster) (note you will need to be authenticated via kerberos on your machine to visit the resourcemanager link) for executors.

First initialize kerberos via a Jupyter terminal. 
In the terminal execute: <BR>
<i>kinit -k -t data/robertop.keytab robertop@CUA.SURFSARA.NL</i><BR>
Print your credentials:


In [1]:
! klist

Ticket cache: FILE:/tmp/krb5cc_1000
Default principal: robertop@CUA.SURFSARA.NL

Valid starting       Expires              Service principal
05/09/2016 08:04:01  05/10/2016 08:04:01  krbtgt/CUA.SURFSARA.NL@CUA.SURFSARA.NL
	renew until 05/09/2016 08:04:01


In [2]:
! hdfs dfs -ls 
execfile('../spark-scripts/bullet.py')

Found 5 items
drwx------   - robertop hdfs          0 2016-05-07 06:00 .Trash
drwxr-xr-x   - robertop hdfs          0 2016-05-09 01:39 .sparkStaging
drwx------   - robertop hdfs          0 2016-04-06 15:54 .staging
drwxr-xr-x   - robertop hdfs          0 2016-04-27 13:07 mattia
drwxr-xr-x   - robertop hdfs          0 2016-04-13 10:00 recsys2016Competition


Verify that we can browse HDFS:

Next initialize Spark. Note that the code below starts a job on the Hadoop cluster that will remain running while the notebook is active. Please close and halt the notebook when you are done. Starting the SparkContext can take a little longer. You can check the YARN resourcemanager to see the current status/usage of the cluster.

In [3]:
import os
import json
os.environ['PYSPARK_PYTHON'] = '/usr/local/bin/python2.7'

HDFS_PATH = "hdfs://hathi-surfsara"
BASE_PATH = HDFS_PATH + '/user/robertop/mattia'

from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
sconf = SparkConf()

# Master is now yarn-client. The YARN and hadoop config is read from the environment
sconf.setMaster("yarn-client")

# You can control many Spark settings via the SparkConf. This determines the amount of executors on the cluster:
sconf.set("spark.executor.instances", "200")
#sconf.set("spark.executor.memory", "20g")

# UFW (firewall) is active on the VM. We explicitly opened these ports and Spark should not bind to random ports:
sconf.set("spark.driver.port", 51800)
sconf.set("spark.fileserver.port", 51801)
sconf.set("spark.broadcast.port", 51802)
sconf.set("spark.replClassServer.port", 51803)
sconf.set("spark.blockManager.port", 51804)
sconf.set("spark.authenticate", True)
sconf.set("spark.yarn.keytab", "/home/jovyan/work/data/robertop.keytab")
sconf.set("spark.yarn.access.namenodes", HDFS_PATH + ":8020")

try:
    sc = SparkContext(conf=sconf)
    sqlCtx = SQLContext(sc) 
    sendNotificationToMattia("Spark Context", "Ready!")
except Exception, err:
    sendNotificationToMattia("Fuck you!", str(err)) 
    print str(err)

# <hr style="clear: both" />

# Now you can run your code

Pick a clustering algorithm (name of the file that provides a classify(x,y [,threshold]) function)

# Load data 

In [13]:
playlistRDD = sc.textFile(BASE_PATH + '/30Mdataset/entities/playlist.idomaar')
playlistIdsRDD = playlistRDD.map(lambda x: json.loads(x.split('\t')[4]))
playlistIdsRDD = playlistIdsRDD.map(lambda x: x['objects'])
total_playlists = playlistIdsRDD.count()
print "Total playlist: " + str(total_playlists)

def filter_bad_entry(x):
    try:
        result = [i['id'] for i in x]
        return (result, 0)
    except:
        return (x, 1)

playlistIdsRDD = playlistIdsRDD.map(filter_bad_entry)
bad_n = playlistIdsRDD.filter(lambda x: x[1] == 1 or len(x[0]) == 0).count()
print "Bad playlists: " + str(bad_n)

playlistOkRDD = playlistIdsRDD.filter(lambda x: x[1] == 0 and len(x[0]))
playlist_count = float(playlistOkRDD.count())
print "Good playlists: " + str(int(playlist_count)) + " (" + str(playlist_count/total_playlists * 100) + "% )"


Total playlist: 57561
Bad playlists: 9139
Good playlists: 48422 (84.1229304564% )


In [19]:
playlistFlatRDD = playlistOkRDD.zipWithIndex().flatMap(lambda x: [(i, x[1]) for i in x[0][0] ] )
print playlistFlatRDD.take(3)
total_songs = playlistFlatRDD.count()
print total_songs

[(3006631, 0), (1885124, 0), (2548942, 0)]
1603040


# Load songs

In [20]:
def my_replace_punct(x):
    ret = ""
    for i in x:
        if i == '+':
            ret += ' '
        else:
            ret += i
    return ret

tracksRDD = sc.textFile(BASE_PATH + '/30Mdataset/entities/tracks.idomaar.gz')
tracksRDD = tracksRDD.repartition(200)
tracksRDD = tracksRDD.map(lambda x: x.split('\t')).map(lambda x: (x[1], json.loads(x[3])['name'].split('/') ) )
tracksRDD = tracksRDD.map(lambda x: (x[0], " ".join( (x[1][0], x[1][2]) ) )).distinct()
tracksRDD = tracksRDD.map(lambda x : (int(x[0]), my_replace_punct(x[1])))
tracksRDD.take(3)

[(u'2740924', u'Talk+Talk Such+A+Shame+(Dub+Mix)'),
 (u'4573230', u'ZZ+Top She+Loves+My+Automobile'),
 (u'2260352', u'P.SUS Live+Today')]

# Join Songs from Playlists with their name

In [25]:
# Join  -> (songID, (playlistID, name))
# Map   -> (playlistID, (songID, name))
# Group -> (playlistID, [list of (songID, name) ] )

playSongsRDD = playlistFlatRDD.join(tracksRDD).map(lambda x: (x[1][0], (x[0], x[1][1])) ).groupByKey()
playSongsRDD.take(3)

[(1048582, (9229, u'Eighteen Nightmares At The Lux Mother Of Girl')),
 (131098, (42518, u'Al Gromer Khan Mahal')),
 (262196, (35822, u'ASP Ich bin ein wahrer Satan (Dope Stars Inc. Remix)'))]

# Co-occurent Song in Playlist with high similarity (NO Duplicate)

In [73]:
CLUSTER_ALGO = 'jaccardBase'
THRESHOLD = 0.751
execfile('../spark-scripts/' + CLUSTER_ALGO + '.py')

def classify_couple(x):
    x_list = list(x[1])
    inserted = set()
    l = len(x_list)
    
    result = []
    
    for i in range(l):
        song_1 = x_list[i]
        id_1 = song_1[0]
        
        for j in range(i):
            song_2 = x_list[j]
            id_2 = song_2[0]
            
            if id_1 == id_2:
                continue
                
            couple_id = (id_1, id_2) if id_1 < id_2 else (id_2, id_1)
            if couple_id in inserted:
                continue
            
            if  song_1[1] != song_2[1] and classify(song_1[1], song_2[1], THRESHOLD):
                couple = (song_1, song_2) if id_1 < id_2 else (song_2, song_1)
                result.append( couple )
            
    return result
            
            

couplesRDD = playSongsRDD.flatMap(classify_couple).distinct()
print couplesRDD.count()
couplesRDD.take(3)

1967


[((678958, u'Coaltar of the Deepers MEXICO'),
  (678959, u'Coaltar of the Deepers Mexico (Ver.1)')),
 ((2782864, u'The Alan Parsons Project Prime Time'),
  (2783061, u'The Alan Parsons Project Time')),
 ((2891042, u'The Jesus and Mary Chain Head'),
  (2891044, u'The Jesus and Mary Chain Head On'))]

# Songs in same cluster non co-occurent -> Duplicates (High probability)

In [45]:
CLUSTER_ALGO = 'jaccardBase'
THRESHOLD = 0.751

clusterSongsFileRDD = sc.pickleFile(BASE_PATH + '/clusters/' + CLUSTER_ALGO + str(THRESHOLD)[2:])
clusterSongsFileRDD = clusterSongsFileRDD.filter(lambda x: len(x[1]) >1)
clusterSongsFileRDD.take(3)


[(3833856, <pyspark.resultiterable.ResultIterable at 0x7f38a7619750>),
 (2232320, <pyspark.resultiterable.ResultIterable at 0x7f38a75601d0>),
 (1540098, <pyspark.resultiterable.ResultIterable at 0x7f38a75600d0>)]

# Song to list of songs in clusters

In [50]:
songToClusterRDD = clusterSongsFileRDD.flatMap(lambda x: [(i, x[1]) for i in x[1]] )
songToClusterRDD = songToClusterRDD.reduceByKey(lambda x, y: list(set(x) | set(y)))
songToClusterRDD.take(10)

[(0, [0, 1937715]),
 (3635200, [3635200, 3635196]),
 (1902400,
  [1902402,
   1902400,
   1902404,
   1902399,
   1902401,
   1902406,
   1902405,
   1902403,
   1902397,
   1902398]),
 (157600, [157601, 157600]),
 (1144800, [1144800, 1144794]),
 (1885200, [1885202, 1885200, 1885201, 1885209, 1885243]),
 (2378800, [2378800, 2378801, 2378802]),
 (1234000, [1233998, 1233999, 1234000]),
 (582800, [582795, 582800]),
 (3502000, [3502001, 3502000])]

# Song to list of songs in playlists

In [59]:
songToPlaylistRDD = playlistOkRDD.flatMap(lambda x: [(i, x[0]) for i in x[0]] ).groupByKey().mapValues(list)
songToPlaylistRDD.take(30)

[(2244608,
  [[1551994,
    3088501,
    2236554,
    278303,
    694674,
    525983,
    3496783,
    3215234,
    694644,
    1492934,
    1373195,
    1242004,
    3720073,
    71174,
    1881602,
    2008132,
    302547,
    2468638,
    2468458,
    2244628,
    2244608,
    2244768]]),
 (1122306,
  [[3620187,
    3620185,
    585736,
    585705,
    585700,
    74535,
    74553,
    3357189,
    3357187,
    2465975,
    2465934,
    2465970,
    3357202,
    3357196,
    3357183,
    912213,
    424351,
    1626999,
    3018759,
    3018769,
    3018761,
    3018765,
    2508217,
    74521,
    2218825,
    2218822,
    2218828,
    2218816,
    2218827,
    2218823,
    2858594,
    2653505,
    2653497,
    2653501,
    1651235,
    1651236,
    1651237,
    2858627,
    2858619,
    2858552,
    2858554,
    2858526,
    2858551,
    2858581,
    2858587,
    2858529,
    2858553,
    2858584,
    912224,
    1281976,
    912318,
    912208,
    912313,
    912188,
    303603

# First check songs with playlists

In [55]:
ids_playlist = songToPlaylistRDD.map(lambda x: x[0]).collect()
ids_playlist

[98308,
 3120632,
 573448,
 953004,
 1400844,
 3682306,
 1671182,
 2457616,
 4842840,
 3089808,
 622612,
 2615982,
 901124,
 983066,
 294944,
 2170914,
 3719204,
 262182,
 1351720,
 3659100,
 213036,
 163886,
 901128,
 1138740,
 1433654,
 3842104,
 1638458,
 1982524,
 524350,
 3047488,
 353632,
 1982530,
 1417284,
 671814,
 114764,
 3113038,
 655440,
 1419960,
 852050,
 1138772,
 3489806,
 2482264,
 1876058,
 360540,
 3721914,
 1114206,
 483344,
 2859106,
 3343718,
 4767846,
 2849468,
 1269866,
 2728044,
 2121838,
 3154032,
 3669474,
 162582,
 1022654,
 3293204,
 1720444,
 106622,
 2953966,
 221316,
 689516,
 352394,
 1319056,
 2437144,
 3522706,
 497662,
 3661972,
 3581294,
 2089112,
 1088196,
 245914,
 3309724,
 1083770,
 319656,
 1564842,
 3487090,
 2425008,
 4833458,
 1212596,
 1871902,
 1605814,
 2051876,
 229560,
 3784890,
 3060626,
 3153952,
 2900162,
 2785476,
 91510,
 2506950,
 531114,
 319688,
 1876170,
 3424460,
 520226,
 2908366,
 1130704,
 614610,
 3752150,
 2179288,
 2117

In [56]:
songClusterPlaylistRDD = songToClusterRDD.join(songToPlaylistRDD)
songClusterPlaylistRDD.take(3)

[(1848396,
  ([1848394, 1848396],
   <pyspark.resultiterable.ResultIterable at 0x7f38a752fc90>)),
 (2182860,
  ([2182860, 2182862],
   <pyspark.resultiterable.ResultIterable at 0x7f38a752f390>)),
 (937464,
  ([937465, 937464],
   <pyspark.resultiterable.ResultIterable at 0x7f38a752f450>))]

# If the songs in cluster are not in the playlists -> generate couple

In [62]:
def cluster_not_in_playlist(x):
    cluster = x[1][0]
    playlist = list(x[1][1])
    result = set()
    
    for i in cluster:
        if i == x[0]:
            continue
        if i in playlist:
            continue
            
        couple = (x[0], i) if x[0] < i else (i, x[0])
        
        result.add(couple)
    
    return list(result)
        
    
couplesDupliRDD = songClusterPlaylistRDD.map(cluster_not_in_playlist).flatMap(lambda x: [i for i in x] )
couplesDupliRDD.take(30)

[(3195900, 3195901),
 (811236, 811237),
 (2114859, 3630462),
 (2261250, 2261251),
 (2130193, 2130198),
 (2130194, 2130198),
 (2130198, 2130199),
 (2130198, 2130222),
 (459082, 459084),
 (303509, 303510),
 (303508, 303510),
 (82403, 82410),
 (1649406, 1649407),
 (529936, 1442376),
 (550338, 550342),
 (49771, 49848),
 (49848, 49849),
 (49772, 49848),
 (1606392, 1606393),
 (2294614, 2294616),
 (1139669, 1139670),
 (2327580, 2327581),
 (486761, 2327580),
 (1197156, 1944615),
 (500732, 500892),
 (3662802, 3663024),
 (607422, 607441),
 (38395, 2360544),
 (1991910, 1991912),
 (2835708, 2835709)]

# Join with track names

In [68]:
couplesFlatRDD = couplesDupliRDD.flatMap(lambda x: [(i, x) for i in x]).join(tracksRDD)
couplesTrackRDD = couplesFlatRDD.map(lambda x: (x[1][0], (x[0], x[1][1])) ).groupByKey().mapValues(set)
couplesTrackRDD.take(3)

[((812381, 812451),
  {(812381, u'Darren Hayes Popular'), (812451, u'Darren Hayes Pop!ular')}),
 ((887013, 887015),
  {(887013, u'Diamond Version Empowering Change'),
   (887015, u'Diamond Version Empowering Change (Version)')}),
 ((1808162, 1808284),
  {(1808162,
    u'Mel Torm%C3%A9 Chestnuts Roasting On An Open Fire (The Christmas Song)'),
   (1808284,
    u'Mel Torm%C3%A9 The Christmas Song (Chestnuts Roasting On An Open Fire)')})]

In [69]:
couplesTrackRDD.take(100)

[((812381, 812451),
  {(812381, u'Darren Hayes Popular'), (812451, u'Darren Hayes Pop!ular')}),
 ((3007470, 3007730),
  {(3007470, u'The White Stripes Hand Springs'),
   (3007730, u'The White Stripes The White Stripes - Hand Springs')}),
 ((1808162, 1808284),
  {(1808162,
    u'Mel Torm%C3%A9 Chestnuts Roasting On An Open Fire (The Christmas Song)'),
   (1808284,
    u'Mel Torm%C3%A9 The Christmas Song (Chestnuts Roasting On An Open Fire)')}),
 ((1042934, 1043038),
  {(1042934, u'Edward Maya 30.Edward Maya - This is My Life'),
   (1043038, u'Edward Maya This Is My Life')}),
 ((1132911, 1132913),
  {(1132911,
    u'Eurythmics There Must Be An Angel (Playing With My Heart) (Live)'),
   (1132913,
    u'Eurythmics There Must Be An Angel (Playing With My Heart) - Remastered Version')}),
 ((2213637, 2213713),
  {(2213637, u'Pixies 07. All Over The World - Pixies'),
   (2213713, u'Pixies All Over the World')}),
 ((1757101, 1757295),
  {(1757101, u'Marvin Gaye (Mercury Edit II) Sunny'),
   (17

In [71]:
print "Duplicates: " + str(couplesTrackRDD.count())
print "Differents: " + str(couplesRDD.count())

Duplicates: 70519
Differents: 178461
